In [1]:
%%configure -f
{"conf": {"spark.jars.packages": "com.databricks:spark-avro_2.11:3.1.0" }}

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("EventDataCapture") \
    .config('spark.hadoop.avro.mapred.ignore.inputs.without.extension', 'false') \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1513931680049_0004,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


//Data from ADLS -> result of Event Data Capture

In [3]:
avroFiles = "adl://adlssparklab.azuredatalakestore.net/mySamples/edcAvro/*.avro"

In [4]:
avroDf = spark.read.format("com.databricks.spark.avro").load(avroFiles)
jsonRdd = avroDf.select(avroDf.Body.cast("string")).rdd.map(lambda x: x[0])

In [5]:
data = spark.read.json(jsonRdd)
print(data)

DataFrame[humidity: bigint, id: bigint, temperature: bigint, timestamp: string, waterLevel: bigint]

In [6]:
data.createOrReplaceTempView("edc")

In [7]:
res = spark.sql("SELECT id,max(waterLevel) FROM edc group by id order by id")

In [8]:
res.show()

+---+---------------+
| id|max(waterLevel)|
+---+---------------+
|  2|             18|
|  4|             57|
|  6|             98|
|  8|             59|
| 10|            113|
+---+---------------+